In [4]:
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 249 kB 9.9 MB/s eta 0:00:01


In [32]:
import pandas as pd 

# Importing Dataset into Dataframe

In [18]:
dataSet = pd.read_excel('data.xlsx')
dataSet.drop(['ModelSize', 'StimType', 'Senttype.wav', 'Senttype', 'Accent', 'WER', 'INDEX'], axis = 1, inplace = True)
dataSet

,File,Predicted,Actual
0,AcF4sent6_nonword.wav,The scarad jewel won't come out of its airplane.,the scared jurtle wont come out of its airplane
1,AcF2sent6_nonword.wav,The scarecrow won't come out of its airplane.,the scared jurtle wont come out of its airplane
2,AcF3sent6_nonword.wav,the scared drill won't come out of its airplane.,the scared jurtle wont come out of its airplane
3,AcF3sent6_nonword.wav,The scared duel won't come out of its airplane.,the scared jurtle wont come out of its airplane
4,AcM2sent6_nonword.wav,The scared girdle won't come out of its airplane.,the scared jurtle wont come out of its airplane
...,...,...,...
43655,AcM4word94_sensical.wav,Zebra.,zebra
43656,AcM2word94_sensical.wav,Zebra.,zebra
43657,AcM1word94_sensical.wav,Zebra.,zebra
43658,enM4word94_sensical.wav,Ziba.,zebra


In [20]:
transcriptions = dataSet
transcriptions.drop(['File', 'Predicted'], axis = 1, inplace = True)
transcriptions

,Actual
0,the scared jurtle wont come out of its airplane
1,the scared jurtle wont come out of its airplane
2,the scared jurtle wont come out of its airplane
3,the scared jurtle wont come out of its airplane
4,the scared jurtle wont come out of its airplane
...,...
43655,zebra
43656,zebra
43657,zebra
43658,zebra


In [29]:
transcriptions['Actual'].map(lambda x: x.lower()) # convert every transcription to lower case
transcriptions

,Actual
0,the scared jurtle wont come out of its airplane
1,the scared jurtle wont come out of its airplane
2,the scared jurtle wont come out of its airplane
3,the scared jurtle wont come out of its airplane
4,the scared jurtle wont come out of its airplane
...,...
43655,zebra
43656,zebra
43657,zebra
43658,zebra


In [34]:
from collections import Counter
import re
column = 'Actual'
combined_text = ''.join(transcriptions[column].dropna()) # combines all text into one and drops NaN values
character_counts = Counter(combined_text)
cleaned_text = re.sub(r"[^a-zA-Z ']", '', combined_text) # removes special characters except for apostrophes and spaces
character_count_dict = dict(character_counts)
character_count_dict["|"] = character_count_dict[" "] # more clear token class for spaces
del character_count_dict[" "]
character_count_dict

{'t': 69485,
 'h': 48095,
 'e': 97005,
 's': 51205,
 'c': 23500,
 'a': 67540,
 'r': 48770,
 'd': 26795,
 'j': 450,
 'u': 19585,
 'l': 33950,
 'w': 21790,
 'o': 62360,
 'n': 44685,
 'm': 17915,
 'f': 15035,
 'i': 47585,
 'p': 16935,
 'b': 19195,
 'y': 15625,
 'g': 20035,
 'q': 675,
 'Q': 10,
 'k': 12775,
 'v': 4755,
 '3': 375,
 'x': 1115,
 'z': 2000,
 '|': 155795}

## Add Padding token that corresponds to CTC "Blank Token"
Read more about CTC here: https://distill.pub/2017/ctc/

In [36]:
character_count_dict["[UNK]"] = len(character_count_dict)
character_count_dict["[PAD]"] = len(character_count_dict)

In [37]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(character_count_dict, vocab_file)

In [38]:
from transformers import Wav2Vec2CTCTokenizer
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")